In [2]:
import json
import requests
import time

In [17]:
with open(r"C:\Users\victo\Git\vbcalinao\boomai-mle\qamodel\train-v2.0.json") as f:
    data = json.load(f)


In [18]:
def get_qa(topic, data):
    q = []
    a = []
    for d in data['data']:
        if d['title']==topic:
            for paragraph in d['paragraphs']:
                for qa in paragraph['qas']:
                    if not qa['is_impossible']:
                        q.append(qa['question'])
                        a.append(qa['answers'][0]['text'])
            return q,a

questions, answers = get_qa(topic='Premier_League', data=data)
print("Number of available questions: {}".format(len(questions)))

Number of available questions: 357


In [19]:
!docker build ../ -t qamodel 


#0 building with "default" instance using docker driver

#1 [internal] load .dockerignore
#1 transferring context: 2B done
#1 DONE 0.0s

#2 [internal] load build definition from Dockerfile
#2 transferring dockerfile: 927B 0.0s done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.8.12-slim-buster
#3 DONE 0.9s

#4 [1/9] FROM docker.io/library/python:3.8.12-slim-buster@sha256:26ab58f6b8936fe59303b0ca0e915d5f9e071ef8b9bf7b3d716b4068f11443dc
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 280B done
#5 DONE 0.0s

#6 [5/9] RUN npm install -g http-server
#6 CACHED

#7 [7/9] RUN bash download_model.sh
#7 CACHED

#8 [3/9] RUN apt-get update && apt-get install -y curl
#8 CACHED

#9 [4/9] RUN curl -fsSL https://deb.nodesource.com/setup_18.x | bash - &&     apt-get install -y nodejs
#9 CACHED

#10 [6/9] COPY download_model.sh .
#10 CACHED

#11 [2/9] RUN apt-get update &&    apt-get install -y --no-install-recommends wget &&    rm -rf /var/lib/apt/lis

In [20]:
!docker run -d -p 8000:8000 qamodel

c0554d46e7ffe1a7838ebdce7a9f2f408d44bf2be685c4d2216cd03610b7bf9c


In [21]:
time.sleep(10)

In [23]:
json_data = {
  'questions':questions,
  'answers':answers,
}

response = requests.post(
  'http://localhost:8000/set_context',
  json=json_data
)

response.json()

{'message': 'Search context set'}

In [45]:
new_questions = [
    'How many clubs?',
    'When does the league starts and finishes ?',
    'Who has the highest number of goals in the Premier League?',
]

json_data = {
  'questions':new_questions,
}

response = requests.post(
  'http://localhost:8000/get_answer',
  json=json_data
)

for d in response.json():
  print('\n'.join(["{} : {}".format(k, v) for k,v in d.items()])+'\n')

orig_q : How many clubs?
best_q : How many club members are there?
best_a : The Premier League is a corporation in which the 20 member clubs act as shareholders

orig_q : When does the league starts and finishes ?
best_q : When does the Premier League have its playing season?
best_a : During the course of a season (from August to May)

orig_q : Who has the highest number of goals in the Premier League?
best_q : Who has the record for most goals in the Premier League?
best_a : Newcastle United striker Alan Shearer holds the record for most Premier League goals with 260

